In [1]:
from random import sample

def factorBerlekamp(f):  # Calcula un factor de f mónico libre de cuadrados sobre
    # un cuerpo finito mediante el algoritmo de Berlekamp.
    if gcd(f,f.derivative()) != 1 or f[f.degree()] != 1:
        print("El polinomio no es libre de cuadrados o no es mónico.")
        return
    BR = f.base_ring()  # Cuerpo finito de los coeficientes de f.
    p = BR.characteristic()
    m = BR.degree()
    q = p ** m
    n = f.degree()
    Rx.<x> = PolynomialRing(BR)
    Phi = []  # Guardará la matriz de a |--> a^q-a.
    for _ in range(n):
        pol = power_mod(x, _ * q, f)  # Cálculo eficiente de x^(iq) mod f
        coef = []  # Guardará los coeficientes.
        if type(pol) == int:
            coef = [pol] + [0] * (n - 1)  # Añadimos ceros hasta n.
        else:
            coef = list(pol)
            lenc = len(coef)
            coef = coef + [0] * (n - lenc)  # Añadimos ceros hasta n.
        Phi.append(coef)
    Phi = matrix(BR, Phi) - matrix.identity(BR, n)
    Ker = Phi.transpose().right_kernel()  # Ker(Phi): Álgebra de Berlekamp.
    BaseKer = Ker.basis_matrix()  # Base de Ker(Phi).
    r = BaseKer.dimensions()[0]  # Número de factores de  f.
    BKer = [Rx(list(_)) for _ in BaseKer]
    if r == 1:  # Si r = 1, es irreducible.
        return f, 1, 1
    while True:  # Repetimos hasta obtener un factor de f.
        a = sample(list(Ker), 1)[0]  # Tomamos a del núcleo pseudoaleatorio.
        a = Rx(list(a))  # Interpretamos a como polinomio.
        b = power_mod(a, (q - 1) // 2, f)  # a ^ ((q - 1) / 2)
        d = gcd(f, b)
        if d != 1 and d != f:
            return d, f // d, r  # Obtenemos un factor de f.
        d = gcd(f, b - 1)
        if d != 1 and d != f:
            return d, f // d, r  # Obtenemos un factor de f.
        
def Berlekamp(f):  # Factorización de f libre de cuadrados sobre 
    # un cuerpo finito con el algoritmo de Berlekamp.
    fact = []  # Guardará los factores de f.
    c = f[f.degree()]
    if c != 1:  # Si f no es mónico, el coeficiente principal es un factor.
        fact = [c]
    g = f // c
    while g.degree() != 0:  # Mientras g no sea constante.
        [d, g, r] = factorBerlekamp(g)  # Aplicamos el algoritmo anterior.
        if r == 1:
            return f
        [d1, g1, r1] = factorBerlekamp(d)  # Buscamos un factor de d.
        [d2, g2, r2] = factorBerlekamp(g)  # Buscamos un factor de g.
        if r1 == 1 and r2 != 1:  # d es irreducible.
            fact.append(d)
        elif r1 != 1 and r2 == 1:  # g es irreducible.
            fact.append(g) 
            g = d
        else:  # d y g son irreducibles.
            fact.append(g)
            fact.append(d)
            return fact

In [2]:
BR.<a> = GF(5)
R.<x> = PolynomialRing(BR)
f = 3 * R(3+2*x+3*x**2+3*x**3+3*x**4+3*x**5)
f = f / 3
print(f)
Berlekamp(f)

3*x^5 + 3*x^4 + 3*x^3 + 3*x^2 + 2*x + 3


[3, x^2 + 4*x + 2, x^3 + 2*x^2 + x + 3]

In [3]:
BR.<a> = GF(3)
R.<x> = PolynomialRing(BR)
f = R(x^4 + 2)
print(f)
Berlekamp(f)

x^4 + 2


[x + 2, x + 1, x^2 + 1]

In [4]:
Berlekamp(f)

[x + 2, x + 1, x^2 + 1]

In [ ]:
# def factoriza(f):
#     BR = f.base_ring()  # Cuerpo finito de los coeficientes de f.
#     p = BR.characteristic()
#     m = BR.degree()
#     q = p ** m
#     n = f.degree()
#     Rx.<x> = PolynomialRing(BR)
#     fact = []
#     h = gcd(f, f.derivative())
#     if h == 1:
#         return list(factorBerlekamp(f))
#     elif h == f:
#         g = f.nth_root(p)
#         return list(factoriza(g))
#     else:
#         fact = [h]
#         fact = fact + factoriza(f // h)
#         return list(fact)